# Setup

In [1]:
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import yaml
import psycopg2
from datetime import datetime
from psycopg2.extras import Json

from langchain_openai import OpenAI
from langchain.globals import set_llm_cache
from langchain_core.caches import BaseCache

In [3]:
with open('config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

In [ ]:
openai = OpenAI(model_name='gpt-3.5-turbo-instruct')

# Se der o erro (Error with OpenAI 1.56.0 - Client.__init__() got an unexpected keyword argument ‘proxies’) ao tentar executar o OpenAI, você reinstalar
# pip install openai==1.55.3 httpx==0.27.2

# Personalizado + Postgres

In [5]:
class PostgresCache(BaseCache):
    def __init__(self, conn):
        self.conn = conn
        self._ensure_table()

    def _ensure_table(self):
        with self.conn.cursor() as cur:
            cur.execute("""
                CREATE TABLE IF NOT EXISTS llm_cache (
                    key TEXT PRIMARY KEY,
                    value JSONB NOT NULL,
                    llm_string TEXT,
                    created_at TIMESTAMP DEFAULT NOW()
                );
            """)
            self.conn.commit()

    def _get_cache_key(self, prompt, llm_string):
        return f"{prompt}|||{llm_string}"

    def lookup(self, prompt, llm_string=None):
        key = self._get_cache_key(prompt, llm_string)
        with self.conn.cursor() as cur:
            cur.execute("SELECT value FROM llm_cache WHERE key = %s;", (key,))
            result = cur.fetchone()
            if result:
                return result[0]  # deve ser uma lista de Generation-like
            return None

    def update(self, prompt, llm_string, return_val):
        key = self._get_cache_key(prompt, llm_string)

        # O LangChain envia uma lista de Generation (mesmo para 1 resultado)
        try:
            generations = [g.text if hasattr(g, "text") else str(g) for g in return_val]
        except Exception as e:
            print("❌ Erro ao processar return_val:", e)
            generations = [str(return_val)]

        value = [{"text": g} for g in generations]

        with self.conn.cursor() as cur:
            cur.execute("""
                INSERT INTO llm_cache (key, value, llm_string, created_at)
                VALUES (%s, %s, %s, %s)
                ON CONFLICT (key) DO UPDATE
                SET value = EXCLUDED.value,
                    llm_string = EXCLUDED.llm_string,
                    created_at = EXCLUDED.created_at;
            """, (key, Json(value), llm_string, datetime.utcnow()))
            self.conn.commit()

    def clear(self, **kwargs):
        with self.conn.cursor() as cur:
            cur.execute("DELETE FROM llm_cache;")
            self.conn.commit()


In [6]:
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="123",
    host="localhost",
    port=5432
)

cache = PostgresCache(conn)
set_llm_cache(cache)
prompt = 'Me diga em poucas palavras quem foi Neil Kal Marx 2.'

In [7]:
def invoke_with_cache_postgres(llm, prompt, cache):
    cached_response = cache.lookup(prompt, "")
    if cached_response:
        print("Usando cache:")
        return cached_response

    response = llm.invoke(prompt)

    # EXTRAI SEMPRE o texto, mesmo que seja um objeto Generation ou outro
    response_text = getattr(response, "text", str(response))

    # GARANTE que o que será salvo no banco é sempre JSON-compatível
    cache.update(prompt, response_text, "")
    return response_text


In [8]:
response1 = invoke_with_cache_postgres(openai, prompt, cache)
response_text1 = response1.replace('\n', ' ') 

print("Primeira resposta (API chamada):", response_text1)

Primeira resposta (API chamada):   Neil Kal Marx foi um filósofo, economista e sociólogo alemão, considerado um dos fundadores do socialismo científico e do comunismo moderno. Suas ideias influenciaram profundamente diversos movimentos políticos e sociais ao redor do mundo. Ele acreditava em uma sociedade igualitária, sem classes sociais e com o controle dos meios de produção nas mãos dos trabalhadores. Seu pensamento revolucionário teve impacto significativo na história do século XX e ainda é objeto de estudo e debate na atualidade. 


In [9]:
response2 = invoke_with_cache_postgres(openai, prompt, cache)
response_text2 = response2.replace('\n', ' ')  

print("Segunda resposta (usando cache):", response_text2)

Segunda resposta (usando cache):   Neil Kal Marx foi um filósofo, economista e sociólogo alemão, considerado um dos fundadores do socialismo científico e do comunismo moderno. Suas ideias influenciaram profundamente diversos movimentos políticos e sociais ao redor do mundo. Ele acreditava em uma sociedade igualitária, sem classes sociais e com o controle dos meios de produção nas mãos dos trabalhadores. Seu pensamento revolucionário teve impacto significativo na história do século XX e ainda é objeto de estudo e debate na atualidade. 
